In [ ]:
#Set up paths
import sys
import os
print( sys.path )

In [ ]:
This_module_path = os.getcwd()  #os.path.dirname(os.path.abspath(__file__))
#workdir_ = os.path.join(This_module_path, '../../' )
workdir_ = '/glade/work/juliob/'
print( f" In {__name__} we have This_module_path={This_module_path} " )
print( f" In {__name__} we have workdir_={workdir_} " )

########################################
"""
sys.path.append(workdir_ + 'myPythonTools/GenlTools/')
sys.path.append(workdir_ + 'myPythonTools/Utils/')
sys.path.append(workdir_ + 'myPythonTools/Plotting/')
sys.path.append(workdir_ + 'myPythonTools/CASutils/')
sys.path.append(workdir_ + 'PyRegridding/Utils/')
sys.path.append(workdir_ + 'PyRegridding/Regridder/')

This_module_path = os.getcwd()  #os.path.dirname(os.path.abspath(__file__))
workdir_ = os.path.join(This_module_path, '../../' )
sys.path.append(workdir_ + 'myPythonTools')
"""
#sys.path.append(workdir_ )
from myPythonTools.Utils import utils as uti
from myPythonTools.Utils import numerical_utils as nuti
from myPythonTools.Utils import AveragingUtils as Av
from myPythonTools.Utils import validation_data as Val
from myPythonTools.Utils import PlotUtil as Pu
from myPythonTools.Plotting import LatLonMaps as LL 
from myPythonTools.CASutils import filter_utils as fu 

from PyRegridding.Utils import GridUtils as GrU
from PyRegridding.Utils import MakePressures as MkP
from PyRegridding.Drivers import RegridField as RgF
#from PyRegridding.Regridder import var_A_x_B as vAB

In [ ]:
# Own local packages

"""
import AveragingUtils as Av
#import VertRegridFlexLL as Vrg  # This is toxic for some reason
import PlotUtil as Pu
import utils as uti
import numerical_utils as nuti
import validation_data as Val
import var_A_x_B as vAB
import MakePressures as MkP
import GridUtils as GrU
import LatLonMaps as LL
import filter_utils as fu

sys.path.append(workdir_ + 'PyRegridding/Drivers/')
import RegridField as RgF
"""


# The usual
from datetime import date
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.tri as tri

# Cartopy for pretty maps
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Some other useful packages 
import importlib
import copy
import time
import cftime
import yaml
import glob
#from box import Box #???

importlib.reload( LL )
importlib.reload( uti )
importlib.reload( nuti )
importlib.reload( Pu )
importlib.reload(Av)
importlib.reload(Val)
#importlib.reload(vAB)
importlib.reload(MkP)
importlib.reload(GrU)


##########################################################
# Read YAML file and open xarray Datasets
#---------------------------------------------------------

with open('configure_plots.yaml', 'r') as file:
    cfg = yaml.safe_load(file)


x='ne240x2_QxQsst'
#x='ne240x2'

exp, subd, Src, Hkey, Dst, useri = cfg[x]['name'] , cfg[x]['subdir'] , cfg[x]['SrcGrid'] , cfg[x]['Hkey'] , cfg[x]['DstGrid'] , cfg[x]['user'] 
ymdPat = "2004-08*" #cfg[x]['ymdPat']

Bdiro = f"/glade/derecho/scratch/juliob/archive/{exp}/atm/GWana/"

Uname=f"{exp}_GWana_U_{ymdPat}.nc"
Vname=f"{exp}_GWana_V_{ymdPat}.nc"
Oname=f"{exp}_GWana_OMEGA_{ymdPat}.nc"
TSname=f"{exp}_GWana_TS_{ymdPat}.nc"


XU = xr.open_mfdataset( Bdiro+Uname )
XV = xr.open_mfdataset( Bdiro+Vname )
XO = xr.open_mfdataset( Bdiro+Oname )


# Define the pattern
pattern = Bdiro+TSname #'dir/file_*'

# Use glob to find all matches
matching_files = glob.glob(pattern)

if matching_files:
    print("We have TS output")
    XTS = xr.open_mfdataset( Bdiro+TSname )
    TS_output=True
else:
    print("TS output not availbale")
    TS_output=False

print( "Got all the data " )
##########################################################
# Generate regirdding objetcs to be usedf later on
#---------------------------------------------------------
try:
    RgOb_ne240_x_ne16
except NameError:
    print("RgOb_ne240_x_ne16 will be created !!!! ")
    RgOb_ne240_x_ne16 = RgF.Horz( Src='ne240pg3', Dst='ne16pg3' ) 
else:
    print("..... RgOb_ne240_x_ne16 has been created")

try:
    RgOb_ne16_x_fv1x1
except NameError:
    print("RgOb_ne16_x_fv1x1 will be created !!!! ")
    RgOb_ne16_x_fv1x1 = RgF.Horz( Src='ne16pg3', Dst='fv1x1',RegridMethod='BILINEAR') 
else:
    print("..... RgOb_ne16_fv1x1 has been created")

try:
    RgOb_ne240_x_llOxO
except NameError:
    print("RgOb_ne240_x_llOxO will be created !!!! ")
    RgOb_ne240_x_llOxO = RgF.Horz( Src='ne240pg3', Dst='latlonOxO' ) 
else:
    print("..... RgOb_ne240_x_llOxO has been created")

##########################################################
# Extract data from xarray Datasets
# Do some calcs
# Make axis variables 
#---------------------------------------------------------

devW = XO.omegaO.values - XO.omegaOx2xO.values
devU = XU.uO.values - XU.uOx2xO.values

devUW = devU * devW 

devUW_c2=RgF.Horz(xfld_Src=devUW , Src='ne240pg3', Dst='ne16pg3' , RegridObj_In=  RgOb_ne240_x_ne16  ) 
devUW_x1=RgF.Horz(xfld_Src=devUW_c2 ,  Src='ne16pg3' , Dst='fv1x1', RegridObj_In=  RgOb_ne16_x_fv1x1  ) 

print( "Finshed with U and OMEGA ")

if (TS_output==True):
    devTS    = XTS.tsO.values -XTS.tsOx2xO.values 
    varTS    = devTS * devTS 
    varTS_c2=RgF.Horz(xfld_Src=varTS , Src='ne240pg3', Dst='ne16pg3' , RegridObj_In=  RgOb_ne240_x_ne16  ) 
    varTS_x1 =RgF.Horz(xfld_Src=varTS_c2 ,  Src='ne16pg3' , Dst='fv1x1', RegridObj_In=  RgOb_ne16_x_fv1x1  ) 
    print( "Finshed with TS ")


clat2,clon2 = GrU.latlon( grid='ne16pg3' )
lat1,lon1 = GrU.latlon( grid='fv1x1' )

days = np.arange( len(XO.time.values) )/4.


##########################################################
# Make fancy plots
#---------------------------------------------------------

fig, ax = plt.subplots(2, 3, figsize=(28, 15))

# Flatten axs to easily access each subplot by index
ax = ax.flatten()

dolev = np.linspace(-1,1,num=41)
olev = np.linspace(-10,10,num=21)

jj = 40


tootrange = f" {(XO.time[0].values.item() ).strftime('%Y-%m-%d')} to {(XO.time[-1].values.item() ).strftime('%Y-%m-%d')}"
dayrange=f"{(XO.time[0].values.item() ).strftime('%Y-%m-%d')}-{(XO.time[-1].values.item() ).strftime('%d')}"
#loot=f"Lat={A.X.lat[jj].values:.1f}"
latval = lat1[jj]
if latval < 0:
    ILat=f'{str(int(-latval))}S'
else:
    ILat=f'{str(int(latval))}N'

print(f" Lat={latval:.1f} ")
print(tootrange )

plev=XO.lev.values
zlev=-7.0*np.log(plev/1_000.)
klist=np.arange( len(zlev) )
p=0
for k in klist:
    print( plev[k] , zlev[k] )
    scak = 500./ plev[k]
    if zlev[k]>30. :
        scak=scak*0.25
    #co=ax[p].contourf( scak*ogOll[0,k,:,:] , levels = 0.5*olev , cmap='bwr' )
    co=ax[p].contourf(lon1, days, scak* devUW_x1[:,k,jj,:],levels=2*dolev ,cmap='bwr' )
    ax[p].set_title( f'{plev[k]:.1f} hPa, {zlev[k]:.1f} km' )
    ax[p].set_ylabel( 'Days' )
    ax[p].set_xlabel( 'Longitude' )
    cb=plt.colorbar( co )
    p=p+1

fig.suptitle( f'{exp}, {tootrange}, Lat={latval:.1f}' , fontsize=20)

FigName=f'Hpass-devUW-xtHovmueller_y{ILat}_{dayrange}_{exp}.png'

print(FigName)
Pu.save_link(png_filename = FigName )

In [ ]:
tsO=XTS.tsO.values

tsO_xO=RgF.Horz(xfld_Src=tsO , Src='ne240pg3', Dst='latlonOxO' , RegridObj_In=  RgOb_ne240_x_llOxO) 
devTS_xO=RgF.Horz(xfld_Src=devTS , Src='ne240pg3', Dst='latlonOxO' , RegridObj_In=  RgOb_ne240_x_llOxO) 


latO,lonO = GrU.latlon( grid='latlonOxO' )

In [ ]:
print(latO[301])

In [ ]:
plt.contourf( lonO, days, devTS_xO[:,301,:] , levels=np.linspace(-1,1,41))

In [ ]:
plt.contourf( lon1, days, varTS_x1[:,40,:] , levels=np.linspace(-1,1,41))

In [ ]:
##########################################################
# Make fancy plots
#---------------------------------------------------------

fig, ax = plt.subplots(2, 3, figsize=(28, 15))

# Flatten axs to easily access each subplot by index
ax = ax.flatten()

dolev = np.linspace(-1,1,num=41)
olev = np.linspace(-10,10,num=21)

jj = 40


tootrange = f" {(XO.time[0].values.item() ).strftime('%Y-%m-%d')} to {(XO.time[-1].values.item() ).strftime('%Y-%m-%d')}"
dayrange=f"{(XO.time[0].values.item() ).strftime('%Y-%m-%d')}-{(XO.time[-1].values.item() ).strftime('%d')}"
#loot=f"Lat={A.X.lat[jj].values:.1f}"
latval = lat1[jj]
if latval < 0:
    ILat=f'{str(int(-latval))}S'
else:
    ILat=f'{str(int(latval))}N'

print(f" Lat={latval:.1f} ")
print(tootrange )

plev=XO.lev.values
zlev=-7.0*np.log(plev/1_000.)
klist=np.arange( len(zlev) )
p=0
for k in klist:
    print( plev[k] , zlev[k] )
    scak = 500./ plev[k]
    if zlev[k]>30. :
        scak=scak*0.25
    #co=ax[p].contourf( scak*ogOll[0,k,:,:] , levels = 0.5*olev , cmap='bwr' )
    co=ax[p].contourf(lon1, days, scak* devUW_x1[:,k,jj,:],levels=2*dolev ,cmap='bwr' )

    #cots = ax[p].contourf(lonO, days, devTS_xO[:,301,:],levels=5*dolev , cmap='bwr' , alpha=0.5 )
    cots = ax[p].contourf(lon1, days, varTS_x1[:,jj,:],levels=5*dolev , cmap='gist_ncar' , alpha=0.25 )
    
    ax[p].set_title( f'{plev[k]:.1f} hPa, {zlev[k]:.1f} km' )
    ax[p].set_ylabel( 'Days' )
    ax[p].set_xlabel( 'Longitude' )
    cb=plt.colorbar( co )
    p=p+1

fig.suptitle( f'{exp}, {tootrange}, Lat={latval:.1f}' , fontsize=20)


In [ ]:
##########################################################
# Make fancy plots
#---------------------------------------------------------

fig, ax = plt.subplots(2, 3, figsize=(28, 15))

# Flatten axs to easily access each subplot by index
ax = ax.flatten()

dolev = 5*np.linspace(0,1,num=41)
olev = np.linspace(-10,10,num=21)

jj = 30


tootrange = f" {(XO.time[0].values.item() ).strftime('%Y-%m-%d')} to {(XO.time[-1].values.item() ).strftime('%Y-%m-%d')}"
dayrange=f"{(XO.time[0].values.item() ).strftime('%Y-%m-%d')}-{(XO.time[-1].values.item() ).strftime('%d')}"
#loot=f"Lat={A.X.lat[jj].values:.1f}"
latval = lat1[jj]
if latval < 0:
    ILat=f'{str(int(-latval))}S'
else:
    ILat=f'{str(int(latval))}N'

print(f" Lat={latval:.1f} ")
print(tootrange )

plev=XO.lev.values
zlev=-7.0*np.log(plev/1_000.)
klist=[0] #np.arange( len(zlev) )
p=0
for k in klist:
    print( plev[k] , zlev[k] )
    scak = 500./ plev[k]
    if zlev[k]>30. :
        scak=scak*0.25
    #co=ax[p].contourf( scak*ogOll[0,k,:,:] , levels = 0.5*olev , cmap='bwr' )
    co=ax[p].contourf(lon1, days, scak* varTS_x1[:,jj,:],levels=dolev ,cmap='bwr' )
    ax[p].set_title( f'{plev[k]:.1f} hPa, {zlev[k]:.1f} km' )
    ax[p].set_ylabel( 'Days' )
    ax[p].set_xlabel( 'Longitude' )
    cb=plt.colorbar( co )
    p=p+1

fig.suptitle( f'{exp}, {tootrange}, Lat={latval:.1f}' , fontsize=20)


In [ ]:
print(np.shape(tsO_xQ))

In [ ]:
varTS_xO=devTS_xO*devTS_xO
rmsTS_xO=np.sqrt( np.average( varTS_xO, axis=0) )

In [ ]:
meanUW_x1 = np.average( devUW_x1, axis=0)  


In [ ]:
#plt.contourf( devTS_xO[10,:,:] ,levels=np.linspace(-1,1,41) )
plt.contourf( lon1, lat1, meanUW_x1[4,:,:] ,levels=np.linspace(0,.2,41) )
plt.colorbar()

In [ ]:
#plt.contourf( devTS_xO[10,:,:] ,levels=np.linspace(-1,1,41) )
plt.contourf( lonO, latO, rmsTS_xO ,levels=np.linspace(-1,1,41) )
plt.colorbar()

In [ ]:
plt.contourf( np.average(varTS_x1,axis=0) ,levels=np.linspace(-1,1,41) )
plt.colorbar()

In [ ]:
plt.contourf( devTS_xO[:,300,:]  , levels=np.linspace(-1,1,41)   )